In [41]:
path = "../runs_RegretNet/X.npy"
import numpy as np
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/home/khanhnq/anaconda3/envs/GAI/lib/python3.9/site-packages/torch/cuda/__init__.py:88: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [42]:
dataset = np.load(path)

In [43]:
dataset.shape

(640000, 5, 3)

In [44]:
import sys
sys.path.append("../")
!ls ..
from core import base
import core.base.base_generator as base_generator
import json
import easydict
config = json.load(open("../runs_RegretNet/setting_default_config/seed_0/config.json", "r"))
config = easydict.EasyDict(config)
print(config)

core  layers   main.py	   plots      requirements.txt	runs_RegretNet	utility
data  LICENSE  mechanisms  README.md  run_configs	target_nets	utils


{'regret_type': 'standard', 'architecture': 'RegretNet', 'distribution_type': 'uniform_01', 'min': 2, 'max': 3, 'num_agents': 5, 'num_items': 3, 'save_data': 'runs_RegretNet/setting_default_config/seed_0', 'dir_name': 'runs_RegretNet', 'plot': {'bool': False, 'n_points': 201}, 'distill': {'architecture': None, 'validate_target_misreports': True, 'train_misreports': True}, 'net': {'init': 'gu', 'activation': 'tanh', 'num_a_layers': 3, 'num_p_layers': 3, 'num_p_hidden_units': 20, 'num_a_hidden_units': 20, 'layer_norm': False, 'hid_att': 16, 'hid': 32, 'n_attention_layers': 1, 'n_attention_heads': 2, 'activation_att': 'tanh', 'pos_enc': False, 'pos_enc_part': 1, 'pos_enc_item': 1, 'n_exch_layers': 3, 'hid_exch': 32, 'activation_exch': 'relu'}, 'train': {'seed': 42, 'restore_iter': 0, 'max_iter': 750, 'learning_rate': 0.001, 'data': 'fixed', 'num_batches': 1250, 'batch_size': 512, 'adv_reuse': True, 'num_misreports': 1, 'gd_iter': 50, 'gd_lr': 0.1, 'gd_lr_step': 1, 'w_rgt_init_val': 1, 'rg

In [45]:
from __future__ import absolute_import, division, print_function

import numpy as np

from core.base.base_generator import BaseGenerator


class Generator(BaseGenerator):
    def __init__(self, config, mode, X=None, ADV=None):
        super(Generator, self).__init__(config, mode)
        self.build_generator(X=X, ADV=ADV)

    def save_data(self, iter):
        return super().save_data(iter)

    def generate_random_X(self, shape):
        return np.random.rand(*shape)

    def generate_random_ADV(self, shape):
        return np.random.rand(*shape)


In [46]:
dataset = Generator(config, "train")

In [47]:
X, ADV, perm = next(dataset.gen_online())
print(X.shape, ADV.shape, perm)

x = torch.from_numpy(X).float().to(device)

(512, 5, 3) (1, 512, 5, 3) None


In [48]:
x.shape

torch.Size([512, 5, 3])